In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json

df_data = pd.read_csv('../data/EnergyMixEU.csv', sep='\t')
display(df_data) 


,Unnamed: 0,renewables,fossil fuels,nuclear
0,Luxembourg,92.80%,7.20%,0.00%
1,Denmark,79.31%,20.69%,0.00%
2,Latvia,75.51%,24.49%,0.00%
3,Lithuania,72.55%,27.45%,0.00%
4,Austria,69.44%,30.56%,0.00%
5,Sweden,69.32%,1.23%,29.45%
6,Croatia,64.64%,35.36%,0.00%
7,Portugal,62.67%,37.33%,0.00%
8,Finland,53.98%,11.00%,35.01%
9,Greece,46.34%,53.66%,0.00%


In [2]:

df_data["renewables"] = df_data["renewables"].str.rstrip("%").astype("float")
df_data["nuclear"] = df_data["nuclear"].str.rstrip("%").astype("float")
df_data["fossil fuels"] = df_data["fossil fuels"].str.rstrip("%").astype("float")
display(df_data)


,Unnamed: 0,renewables,fossil fuels,nuclear
0,Luxembourg,92.80,7.20,0.00
1,Denmark,79.31,20.69,0.00
2,Latvia,75.51,24.49,0.00
3,Lithuania,72.55,27.45,0.00
4,Austria,69.44,30.56,0.00
5,Sweden,69.32,1.23,29.45
6,Croatia,64.64,35.36,0.00
7,Portugal,62.67,37.33,0.00
8,Finland,53.98,11.00,35.01
9,Greece,46.34,53.66,0.00


In [3]:

df_data.columns.values[0] = "Country"
df_data.rename(columns={'fossils fuels': 'Fossil'}, inplace=True)
df_data.rename(columns={'renewables': 'Renewable'}, inplace=True)
df_data.rename(columns={'nuclear': 'Nuclear'}, inplace=True)

highcharts_series = []
for column in df_data.columns:
    series_data = {
        'name': column,
        'data': df_data[column].fillna(0).tolist()  # Replace NaN with 0 for Highcharts
    }
    highcharts_series.append(series_data) 
    
with open('../public/data/energiemix_eu.json', 'w') as json_file:
    json.dump(highcharts_series, json_file, indent=2)